In [1]:
# Implementation for Hedge Backpropagation Feed Forward Network for Classification
# Original code in Theano https://github.com/LIBOL/ODL
# Paper https://www.ijcai.org/proceedings/2018/369
#
# By Carlos Muniz Cuza and Jonas Brusokas

In [2]:
from sail.models.torch.onn_hbp import ONNHBP_Classifier, _ONNHBPModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.datasets import load_iris
from sklearn.utils import gen_batches
import numpy as np
import torch

### 1. Create dataset for classification

In [3]:
n_data_points = 40
n_features = 15
n_classes = 5

X, y = make_classification(n_samples=n_data_points,
                           n_features=n_features,
                           n_informative=n_classes,
                           random_state=0,
                           n_classes=n_classes,
                           n_clusters_per_class=1)

### 2. Import and check that the model works.

In [4]:
ffn_hidden_units = 16
n_hidden_layers = 2

model = _ONNHBPModel(input_units=n_features,
                     output_units=n_classes,
                     hidden_units=ffn_hidden_units,
                     n_hidden_layers=n_hidden_layers)

output = model(X)
print(output)

tensor([[0.1876, 0.2162, 0.2188, 0.1787, 0.1988],
        [0.1603, 0.2455, 0.3031, 0.1272, 0.1639],
        [0.2113, 0.2127, 0.2068, 0.1876, 0.1816],
        [0.1605, 0.2464, 0.2819, 0.1386, 0.1726],
        [0.1980, 0.2183, 0.2016, 0.1971, 0.1850],
        [0.1320, 0.2644, 0.3593, 0.1044, 0.1399],
        [0.2118, 0.2232, 0.1889, 0.1943, 0.1817],
        [0.1667, 0.2385, 0.2793, 0.1426, 0.1730],
        [0.1712, 0.2395, 0.2612, 0.1558, 0.1723],
        [0.2047, 0.2089, 0.2163, 0.1795, 0.1905],
        [0.1681, 0.2464, 0.2781, 0.1406, 0.1668],
        [0.1663, 0.2340, 0.2662, 0.1499, 0.1835],
        [0.1666, 0.2340, 0.2578, 0.1598, 0.1818],
        [0.2078, 0.2543, 0.1879, 0.1739, 0.1761],
        [0.1834, 0.2412, 0.1957, 0.1948, 0.1849],
        [0.1718, 0.2702, 0.2565, 0.1437, 0.1578],
        [0.2029, 0.2240, 0.1931, 0.1915, 0.1885],
        [0.1740, 0.2048, 0.2773, 0.1525, 0.1914],
        [0.2045, 0.2169, 0.1945, 0.2029, 0.1812],
        [0.1794, 0.2088, 0.2996, 0.1344, 0.1778],


### 3. Check skorch

In [5]:
model_skorch = ONNHBP_Classifier(input_units=n_features,
                                 output_units=n_classes,
                                 hidden_units=ffn_hidden_units,
                                 n_hidden_layers=n_hidden_layers)

partial_fit = model_skorch.partial_fit(X,y)
print(partial_fit)
predict = model_skorch.predict(X)
print(predict)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.6122       0.5000        1.5496  0.0125
<class 'sail.models.torch.onn_hbp.ONNHBP_Classifier'>[initialized](
  module_=_ONNHBPModel(
    (hidden_layers): ModuleList(
      (0): Linear(in_features=15, out_features=16, bias=True)
      (1): Linear(in_features=16, out_features=16, bias=True)
    )
    (output_layers): ModuleList(
      (0): Linear(in_features=16, out_features=5, bias=True)
      (1): Linear(in_features=16, out_features=5, bias=True)
    )
    (do): Dropout(p=0.2, inplace=False)
    (actfn): ReLU()
  ),
)
[0 0 2 1 0 1 0 1 1 1 0 1 1 1 0 0 4 2 1 2 1 0 2 0 1 2 0 2 1 0 0 1 0 1 1 1 0
 2 0 1]


### 2. Load the Iris dataset

In [6]:
iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

# Scale data to have mean 0 and variance 1
# which is importance for convergence of the neural network
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data set into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

### 3. Train and test ONN on Iris dataset

In [7]:
n_features = X_train.shape[1]
n_classes = np.unique(y_test).shape[0]
ffn_hidden_units = 50
n_hidden_layers = 3

model_skorch = ONNHBP_Classifier(input_units=n_features,
                                 output_units=n_classes,
                                 hidden_units=ffn_hidden_units,
                                 n_hidden_layers=n_hidden_layers)
partial_fit = None
for i in range(0,25):
    partial_fit = model_skorch.partial_fit(X_train, y_train)

print('Accuracy on the test data', partial_fit.score(X_test, y_test))

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.0555       0.6667        1.0567  0.0160
      2        1.0543       0.6667        1.0552  0.0189
      3        1.0527       0.6667        1.0536  0.0160
      4        1.0512       0.6667        1.0521  0.0179
      5        1.0496       0.6667        1.0506  0.0206
      6        1.0481       0.6667        1.0491  0.0167
      7        1.0465       0.6667        1.0476  0.0190
      8        1.0450       0.6667        1.0461  0.0160
      9        1.0435       0.6667        1.0446  0.0110
     10        1.0420       0.6667        1.0431  0.0080
     11        1.0404       0.6667        1.0416  0.0100
     12        1.0389       0.6667        1.0402  0.0070
     13        1.0374       0.6667        1.0387  0.0080
     14        1.0360       0.6667        1.0372  0.0080
     15        1.0345       0.6667        1.0358  0.0095
     16        1.0330       0.6

### 4. Mini-batch training.

In [8]:
batch_size = 20

model_skorch = ONNHBP_Classifier(input_units=n_features,
                                 output_units=n_classes,
                                 hidden_units=ffn_hidden_units,
                                 n_hidden_layers=n_hidden_layers)

for batch in gen_batches(X_train.shape[0], batch_size):
    x_batch = X_train[batch]
    y_batch = y_train[batch]
    partial_fit = model_skorch.partial_fit(x_batch, y_batch)

predict = model_skorch.predict(X_test)

print(partial_fit.score(X_test, y_test))

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.1026       0.2500        1.1016  0.0060
      2        1.0883       0.5000        1.0336  0.0050
      3        1.0866       0.2500        1.1366  0.0060
      4        1.0908       0.5000        1.0851  0.0059
      5        1.1273       0.0000        1.1302  0.0059
      6        1.0656       0.5000        1.0716  0.0058
0.6


D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


### 5. Improving the results.
Note, the results of doing mini0batch learning are very bad. This is because we only do one single epoch. An easy way to improve this is by running partial fit for each mini-batch several times

In [9]:
for _ in range(10): # n_epochs
    for batch in gen_batches(X_train.shape[0], batch_size):
        x_batch = X_train[batch]
        y_batch = y_train[batch]
        partial_fit = model_skorch.partial_fit(x_batch, y_batch)
        # Shuffling the dataset
        permutation = torch.randperm(X_train.shape[0])
        X_train = X_train[permutation]
        y_train = y_train[permutation]

# Note how the results improved considerably
print(partial_fit.score(X_test, y_test))

      7        1.0954       0.5000        1.0910  0.0069
      8        1.1192       0.0000        1.1396  0.0059
      9        1.0902       0.5000        1.0810  0.0070
     10        1.0526       0.5000        1.1153  0.0060
     11        1.0768       0.5000        1.0667  0.0080
     12        1.0903       0.7500        1.0621  0.0082
     13        1.0912       0.2500        1.0717  0.0070
     14        1.0583       0.5000        1.0440  0.0070
     15        1.1026       0.7500        1.0735  0.0060
     16        1.0826       0.5000        1.0741  0.0070
     17        1.0729       0.7500        1.0332  0.0060
     18        1.0815       0.5000        1.1322  0.0060


D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


     19        1.0465       0.7500        1.0499  0.0060
     20        1.0818       0.7500        1.0793  0.0065
     21        1.0596       1.0000        1.0293  0.0060
     22        1.0525       1.0000        1.0075  0.0070
     23        1.0527       0.7500        1.0611  0.0070
     24        1.0360       0.7500        1.0442  0.0070
     25        1.0555       0.7500        1.0192  0.0070
     26        1.0436       0.7500        1.0267  0.0060


D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least pop

     27        1.0714       0.5000        1.0519  0.0055
     28        1.0734       0.5000        1.0562  0.0067
     29        1.0796       0.5000        1.1286  0.0081
     30        1.0129       1.0000        0.9794  0.0097
     31        1.0313       0.5000        1.0667  0.0209
     32        1.0562       0.5000        1.0653  0.0199
     33        1.0304       0.7500        1.0391  0.0179


...

D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


     55        0.9970       1.0000        0.9553  0.0160
     56        1.0108       0.5000        1.0694  0.0140
     57        0.9908       0.7500        0.9747  0.0159
     58        1.0323       0.5000        1.0452  0.0138
     59        1.0248       0.7500        1.0021  0.0130
     60        1.0011       1.0000        0.9386  0.0170
     61        1.0405       0.7500        0.9789  0.0140
     62        1.0130       0.5000        1.0586  0.0150


D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


     63        0.9748       0.7500        1.0045  0.0168
     64        0.9869       0.7500        0.9982  0.0140
     65        1.0123       0.7500        0.9966  0.0162
     66        0.9900       0.7500        0.9970  0.0160
0.7333333333333333


D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


### 6. Testing on big datasets:
We have downloaded Higgs dataset and store in ../../datasets/higgs.npz.

> https://drive.google.com/drive/folders/1fNZHK2NYbgfz27PPdSSA6lZTkoFakH28

In this example, we only use a subset of higgs. The full dataset has millions of examples.

In [10]:
import itertools


data = np.load('../../datasets/higgs.npz')
data_values = list(itertools.islice(data.values(), 2))
x_values = data_values[0]
y_values = data_values[1]

_, X_sub, _, y_sub = train_test_split(x_values, y_values, test_size=0.01, random_state=42, stratify=y_values)

X_train, X_test, y_train, y_test = train_test_split(X_sub, y_sub, test_size=0.3, random_state=42, stratify=y_sub)

y_train = np.argmax(y_train, axis=1) # Fixing the format to skorch
y_test = np.argmax(y_test, axis=1)

print('Number of training examples', X_train.shape)
print('Number of training examples', X_test.shape)

Number of training examples (35000, 28)
Number of training examples (15000, 28)


In [11]:

n_hidden_layers = 19
ffn_hidden_units =  100
n_features = 28
n_classes = 2
batch_size = 20

model_skorch = ONNHBP_Classifier(input_units=n_features,
                                 output_units=n_classes,
                                 hidden_units=ffn_hidden_units,
                                 n_hidden_layers=n_hidden_layers)

for batch in gen_batches(X_train.shape[0], batch_size):
        x_batch = X_train[batch]
        y_batch = y_train[batch]
        partial_fit = model_skorch.partial_fit(x_batch, y_batch)



  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6947       0.7500        0.6929  0.1436
      2        0.6955       0.5000        0.6985  0.1121
      3        0.6980       0.7500        0.6905  0.1085
      4        0.6994       0.2500        0.6982  0.1122
      5        0.6970       0.2500        0.7011  0.1029
      6        0.6964       0.7500        0.6872  0.1087
      7        0.6926       0.7500        0.6949  0.1057
      8        0.6935       0.7500        0.6846  0.1067
      9        0.6945       0.2500        0.6977  0.1068
     10        0.6955       0.7500        0.6889  0.1078
     11        0.6983       0.7500        0.6905  0.1087
     12        0.7005       0.0000        0.7070  0.1042


D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


     13        0.6951       0.2500        0.6944  0.1091
     14        0.6912       0.7500        0.6880  0.1067
     15        0.6936       1.0000        0.6872  0.1081
     16        0.6957       0.5000        0.6938  0.1078
     17        0.6897       0.7500        0.6856  0.1087
     18        0.6948       0.5000        0.6974  0.1027
     19        0.6962       0.5000        0.7040  0.1077
     20        0.6951       0.5000        0.6939  0.1145
     21        0.7036       0.0000        0.7070  0.0972


D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


     22        0.6967       0.7500        0.6915  0.1076
     23        0.6879       0.2500        0.7013  0.1087
     24        0.6972       0.5000        0.6964  0.1016
     25        0.6965       0.2500        0.7024  0.1083
     26        0.6970       0.5000        0.6930  0.1076
     27        0.6994       0.2500        0.6997  0.0984
     28        0.6932       0.2500        0.7074  0.0995
     29        0.7032       0.0000        0.7084  0.1153
     30        0.6972       0.2500        0.6995  0.1047
     31        0.6921       0.5000        0.6863  0.1041
     32        0.6981       0.2500        0.7030  0.1039
     33        0.6962       0.5000        0.6932  0.1018
     34        0.6996       0.7500        0.6901  0.1067
     35        0.6946       0.5000        0.6980  0.1042
     36        0.6968       0.5000        0.6855  0.1081
     37        0.6971       0.5000        0.6938  0.1087
     38        0.6939       0.5000        0.6939  0.1133
     39        0.6963       0.2

...

   1432        0.6855       0.5000        0.6870  0.1207
   1433        0.6907       0.7500        0.6936  0.1154
   1434        0.6922       0.7500        0.6775  0.1158
   1435        0.6929       0.7500        0.6838  0.1131
   1436        0.6921       0.5000        0.6865  0.1217
   1437        0.6853       0.7500        0.6845  0.1187
   1438        0.6938       0.7500        0.6855  0.1242
   1439        0.6860       0.7500        0.6898  0.1142
   1440        0.6898       0.7500        0.6835  0.1119
   1441        0.6965       0.5000        0.6884  0.1279
   1442        0.6893       0.5000        0.6970  0.1121
   1443        0.6909       0.7500        0.6833  0.1143
   1444        0.6999       0.2500        0.7014  0.1192
   1445        0.6851       1.0000        0.6764  0.1147
   1446        0.6837       0.7500        0.6848  0.1129
   1447        0.6891       0.5000        0.6916  0.1102
   1448        0.6900       0.7500        0.6877  0.1107
   1449        0.6881       0.2

D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


   1503        0.6850       0.7500        0.6753  0.1088
   1504        0.6976       0.2500        0.7088  0.1167
   1505        0.6892       0.7500        0.6836  0.1137
   1506        0.6986       0.2500        0.7003  0.1137
   1507        0.6949       0.5000        0.6936  0.1083
   1508        0.6870       0.5000        0.6983  0.1218
   1509        0.6851       0.7500        0.6671  0.1187
   1510        0.6971       0.2500        0.7019  0.1197
   1511        0.6927       0.2500        0.7080  0.1207
   1512        0.6841       0.7500        0.6841  0.1305
   1513        0.6972       0.5000        0.6976  0.1281
   1514        0.6926       0.5000        0.6899  0.1289
   1515        0.7015       0.5000        0.6946  0.1097
   1516        0.6948       0.5000        0.6916  0.1088
   1517        0.6929       0.2500        0.7003  0.1078
   1518        0.6936       0.5000        0.6976  0.1067
   1519        0.6824       0.7500        0.6811  0.1032
   1520        0.6903       0.7

D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


   1666        0.7023       0.2500        0.7054  0.1083
   1667        0.6919       0.7500        0.6884  0.1087
   1668        0.6974       0.5000        0.6858  0.1126
   1669        0.6925       0.5000        0.6900  0.1155
   1670        0.6914       0.5000        0.6927  0.1162
   1671        0.6953       0.5000        0.6946  0.1217
   1672        0.6934       0.2500        0.7039  0.1134
   1673        0.6875       0.7500        0.6837  0.1161
   1674        0.6933       0.7500        0.6909  0.1146
   1675        0.6933       0.5000        0.6947  0.1127
   1676        0.7018       0.5000        0.6884  0.1110
   1677        0.6947       0.2500        0.7092  0.1207
   1678        0.6835       0.7500        0.6828  0.1161
   1679        0.6887       0.7500        0.6833  0.1230
   1680        0.6846       0.7500        0.6830  0.1197
   1681        0.6946       0.2500        0.7038  0.1199
   1682        0.6874       0.7500        0.6845  0.1037
   1683        0.6868       0.5